In [1]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools

df=pd.read_csv("palermo_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [6]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))
len(coppie_df)


15

In [4]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe 1"
    else: prezzi_inferiori = "Dataframe 2"
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len
    dict_df['tipo_ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_differenze = pd.DataFrame(list_dict_df)   
df_differenze.sort_values(by=['num_differenze_prezzo','media'],ascending=False)


,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori
9,03-03-2024---17:20 *** 03-03-2024---17:44,339,10.672566,8.0,0.347336,desktop/mobile,Dataframe 2
2,03-03-2024---16:59 *** 03-03-2024---17:44,337,8.762611,7.0,0.345287,desktop/mobile,Dataframe 2
6,03-03-2024---17:10 *** 03-03-2024---17:44,329,10.328267,8.0,0.347413,desktop/mobile,Dataframe 2
3,03-03-2024---16:59 *** 03-03-2024---17:59,321,9.778816,8.0,0.328893,desktop/mobile,Dataframe 2
11,03-03-2024---17:20 *** 03-03-2024---18:30,320,10.696875,8.0,0.328542,desktop/mobile,Dataframe 2
4,03-03-2024---16:59 *** 03-03-2024---18:30,315,9.987302,8.0,0.323409,desktop/mobile,Dataframe 2
8,03-03-2024---17:10 *** 03-03-2024---18:30,308,10.318182,8.0,0.326271,desktop/mobile,Dataframe 2
10,03-03-2024---17:20 *** 03-03-2024---17:59,304,10.220395,8.0,0.311475,desktop/mobile,Dataframe 2
7,03-03-2024---17:10 *** 03-03-2024---17:59,294,9.867347,8.0,0.310454,desktop/mobile,Dataframe 2
13,03-03-2024---17:44 *** 03-03-2024---18:30,110,8.309091,6.0,0.109127,mobile/mobile,Dataframe 1


In [ ]:
# Elimino una o più ricerche dal dataframe
new_df = df_differenze.drop(df_differenze[df_differenze['coppia_ricerche'].str.contains("20:42")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

In [7]:
#Visualizza dataframe mobile/mobile o desktop/desktop
df_tipo_uguale = df_differenze[(df_differenze['tipo_ricerche']=="mobile/mobile") | (df_differenze['tipo_ricerche']=="desktop/desktop")]
df_tipo_uguale.sort_values(by="num_differenze_prezzo",ascending= False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori
13,03-03-2024---17:44 *** 03-03-2024---18:30,110,8.309091,6.0,0.109127,mobile/mobile,Dataframe 1
14,03-03-2024---17:59 *** 03-03-2024---18:30,86,10.151163,8.0,0.085317,mobile/mobile,Dataframe 1
0,03-03-2024---16:59 *** 03-03-2024---17:10,65,11.107692,9.0,0.072383,desktop/desktop,Dataframe 1
1,03-03-2024---16:59 *** 03-03-2024---17:20,65,12.476923,9.0,0.066124,desktop/desktop,Dataframe 1
12,03-03-2024---17:44 *** 03-03-2024---17:59,58,8.793103,6.5,0.057540,mobile/mobile,Dataframe 1
5,03-03-2024---17:10 *** 03-03-2024---17:20,19,12.000000,9.0,0.020971,desktop/desktop,Dataframe 1


In [8]:
mobile = 0
desktop = 0
for i in df_list:
    orario1 = i
    df1 = df[df["TIMESTAMP"]==orario1]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print(mobile,desktop)

['desktop']
['desktop']
['desktop']
['mobile']
['mobile']
['mobile']
3 3
